In [ ]:
!pip install crccheck

STAMPA SU UN FILE TUTTI I PATH DEL DATASET


In [ ]:
import os 
if not os.path.exists("all_path.txt"):
  !find . -type d -exec sh -c '(ls -p "{}"|grep />/dev/null)||echo "{}"' \; >> all_path.txt

In [3]:
dominio = input("Inserisci Dominio da analizzare:\n")

Inserisci Dominio da analizzare:
study


INIZIO PRIMO PASSO

In [ ]:
import os
import pickle
import codecs
import re
import random 
from bs4 import BeautifulSoup
from crccheck.crc import Crc8

def create_coeff_random(p):
    v_a = []
    v_b = []
    for i in range(8):   
      v_a.append(random.randint(1,(p-1)))
      v_b.append(random.randint(0,(p-1)))
    return v_a,v_b

def isPrime(n) : 
    if (n <= 1) : 
        return False
    if (n <= 3) : 
        return True
    if (n % 2 == 0 or n % 3 == 0) : 
        return False
    i = 5
    while(i * i <= n) : 
        if (n % i == 0 or n % (i + 2) == 0) : 
            return False
        i = i + 6
    return True

# h(x) = (ax + b) % p
# dove a è un numero random > 0; 
# dove x è ogni vettore dello shingle
# dove b numero random >= 0
# a,b < p
# dove p è un numero primo compreso tra U e 2U dove U=2^32
def calcola_shingles_v(shingle,c_a,c_b): 
  v_8byte_min = []
  for i in range(8):
    
    v_app = [] 
    a = c_a[i]
    b = c_b[i]
    for element in shingle:
      v_app.append(((a * element) + b)%p)
    v_8byte_min.append(min(v_app))
  return v_8byte_min

if not os.path.exists(dominio+"_v_soap.pkl") :

  dominio_v = {}
  f_dominio_v = open(dominio+"_v_soap.pkl","wb")

  U = 2**32
  p = random.randint(U,2*U)
  while not (isPrime(p)):
    p = random.randint(U,2*U)

  coeffs_a,coeffs_b = create_coeff_random(p)

  f = open("all_path.txt","r")
  all_path = f.readlines()

  for path in all_path:
    pages = os.listdir(path[:-1])
    if dominio in path:
      for page in pages:
            tagList = []
            #print(path[:-1]+"/"+page)
            html = open(path[:-1]+"/"+page).read()
            soup = BeautifulSoup(html, "html.parser")
            tags = soup.find_all() 
            for tag in tags :
              tagList.append(tag.name)              
            key = (path[:-1]+"/"+page)[11:-1]
            #print(key)
            shingles = []
            l = 10 #length window
            for i in range(len(tagList)):
              if ((i+l)<=len(tagList)):
                s = ' '.join(tagList[i:i+l])
                crc = Crc8.calc(s.encode('utf-8'))
                shingles.append(crc)
              else:
                break

            v = calcola_shingles_v(shingles,coeffs_a,coeffs_b)
            dominio_v[str(key)] = v
            key = ""

  pickle.dump(dominio_v,f_dominio_v)
  f_dominio_v.close()



In [ ]:
import pickle
import os

def create_masked_shingle(vector):  
  v_8 = vector #vettore 8/8

  v_7 = [] #lista di vettori 7/8
  for i in range(len(vector)):
    v7_app = [None]*len(vector)
    v7_app[i] = "*"
    for j in range(len(vector)):
      if i!=j:
        v7_app[j] = vector[j]
    v_7.append(v7_app)

  v_6 = [] #lista di vettori 6/8
  for i in range(len(vector)):
    for j in range(len(vector)):
      v6_app = ["*"]*len(vector)
      if (j<i) :
        v6_app[j] = vector[j]
      elif (j==i) : 
        v6_app[j] = "*"
      else :
        v6_app[j]="*"
        for k in range(len(vector)): 
          if (k != i and k != j ) :
            v6_app[k]=vector[k]
        v_6.append(v6_app)

  return v_6,v_7,v_8

if not os.path.exists("hash_table_"+dominio+"_soap.pkl") :

  dominio_to_read = open(dominio+"_v_soap.pkl", "rb")
  
  page_v = pickle.load(dominio_to_read)
  hash_table = {}
  for key in page_v.keys():
    v = page_v.get(key)
    v_6,v_7,v_8 = create_masked_shingle(v)     
    for masked_v6 in v_6:
      if str(masked_v6) in hash_table.keys():
        value = hash_table.get(str(masked_v6)) 
        value += 1
        hash_table[str(masked_v6)] = value
      else : 
        hash_table[str(masked_v6)] = 1

    for masked_v7 in v_7:
      if str(masked_v7) in hash_table.keys():
        value = hash_table.get(str(masked_v7)) 
        value += 1
        hash_table[str(masked_v7)] = value
      else : 
        hash_table[str(masked_v7)] = 1
    
    
    if str(v_8) in hash_table.keys():
      value = hash_table.get(str(v_8)) 
      value += 1
      hash_table[str(v_8)] = value
    else : 
      hash_table[str(v_8)] = 1

  f_hash_dominio_soap = open("hash_table_"+dominio+"_soap.pkl", "wb")
  pickle.dump(hash_table,f_hash_dominio_soap)
  f_hash_dominio_soap.close()




INIZIO SECONDO PASSO

In [ ]:
import pickle 

f_hash_dominio_soap = open("hash_table_"+dominio+"_soap.pkl", "rb")

hash_table = pickle.load(f_hash_dominio_soap)
# prendi chiavi 8/8 e ordinale in modo crescente in base al valore (conteggio)
sort_keys_8_8 = sorted(hash_table.items(), key=lambda x: x[1])
sorted_8_8 = {}
for i in sort_keys_8_8:
  if not "*" in i[0]:
    sorted_8_8[str(i[0])] = i[1]

f_sorted_dominio_soap = open("sorted_hash_table_"+dominio+"_soap.pkl", "wb")
pickle.dump(sorted_8_8,f_sorted_dominio_soap)
f_sorted_dominio_soap.close()

  



In [ ]:
def to_list(s):
  s_to_list = []

  char_next= None
  char_before= None
  index_2 = 0
  
  for index, char in enumerate(s):
    
    if (index!=index_2):      
      char = new_char     
      index= index_2
    else:
      index_2+=1

    if char == "*":
      s_to_list.append(char)

    if char.isnumeric(): 
      number = char
      char_next = s[index+1] 
      while char_next.isnumeric():
        index=index+1
        number += char_next
        char_next= s[index+1]         
      else:
        new_index = index+1
        index_2= new_index
        new_char= char_next
        s_to_list.append(number) 

  return s_to_list

In [ ]:
import os
if not os.path.exists("f_"+dominio+"_hash_soap_update.pkl") :
  sorted_8_8_dominio = open("sorted_hash_table_"+dominio+"_soap.pkl", "rb")

  sorted_8_8 = pickle.load(sorted_8_8_dominio)
  for k in sorted_8_8.keys():
    all_candidated = {}
    key_max = [None,0]
    for k1 in hash_table.keys():
      #if "*" in k1:
      contatore = 0
      k_to_list = to_list(k)
      k1_to_list = to_list(k1)

      for i in range(len(k_to_list)): #k e k1 lunghi uguale
        if (k_to_list[i]==k1_to_list[i] or k1_to_list[i]=="*"):
          contatore += 1
      if contatore == 8:
        all_candidated[str(k1)] = hash_table.get(str(k1))
        if int(all_candidated.get(str(k1))) > key_max[1] :
          key_max[0] = str(k1)
          key_max[1] = all_candidated.get(str(k1))

    if str(key_max[0]) in  all_candidated.keys():
      del all_candidated[str(key_max[0])]
    for candidate_k in all_candidated.keys():
      hash_table[candidate_k] = all_candidated.get(candidate_k) - sorted_8_8.get(k) # il primo è uguale al valore che troveremo nell'hash table

  f_dominio_hash_soap_update = open("f_"+dominio+"_hash_soap_update.pkl", "wb")
  pickle.dump(hash_table,f_dominio_hash_soap_update)
  f_dominio_hash_soap_update.close()

    

In [ ]:
import os
import pickle 

if not os.path.exists("f_"+dominio+"_h_support.pkl"):

  f_dominio_hash_soap_update = open("f_"+dominio+"_hash_soap_update.pkl", "rb")
  
  if "guide" in dominio :
    threshold = 10
  else:
    threshold = 20

  h_support = {}
  hash_table = pickle.load(f_dominio_hash_soap_update)
  for k in hash_table.keys():
    if not hash_table.get(k) < threshold:
      h_support[str(k)] = hash_table.get(k)

  f_dominio_h_support = open("f_"+dominio+"_h_support.pkl", "wb")
  pickle.dump(h_support,f_dominio_h_support)
  f_dominio_h_support.close()


TERZO PASSO

In [ ]:
import numpy as np
from collections import defaultdict 
import pickle 

if not os.path.exists(dominio+"_clusters.pkl") :

  dominio_v_pickle = open(dominio+"_v_soap.pkl", "rb")
  all_v = pickle.load(dominio_v_pickle)

  dominio_support = open("f_"+dominio+"_h_support.pkl", "rb")
  h_support = pickle.load(dominio_support)


  clusters = defaultdict(list) 

  for p in all_v.keys(): #le chiavi sono le pagine, i valore le v

    all_mask = {}
    key_max = [None,0]
    for h_key in h_support.keys(): #le chiavi sono le v (maskered), i valori i conteggi
      contatore = 0
      value_p = all_v.get(p)
      list_value_p = to_list(str(value_p))
      list_h_key = to_list(str(h_key))
      for i in range(len(list_value_p)):
        if list_h_key[i] == "*" or list_h_key[i] == list_value_p[i] :
          contatore += 1
      if contatore == 8:
        all_mask[str(h_key)] = h_support.get(str(h_key))
        if all_mask.get(str(h_key)) > key_max[1] :
          key_max[0] = str(h_key)
          key_max[1] = all_mask.get(str(h_key))
    if not str(key_max[0]) == "None":
      clusters[str(key_max[0])].append(p)

  print(clusters)

  dominio_clusters = open(dominio+"_clusters.pkl", "wb")
  pickle.dump(clusters,dominio_clusters)
  dominio_clusters.close()



defaultdict(<class 'list'>, {"['*', 28031325, 56395195, 60911675, 39234713, '*', 18715876, 34537133]": ['www.study.eu/country/franc', 'www.study.eu/country/austri', 'www.study.eu/country/belgiu', 'www.study.eu/country/czech-republi', 'www.study.eu/country/denmar', 'www.study.eu/country/estoni', 'www.study.eu/country/faroe-island', 'www.study.eu/country/finlan', 'www.study.eu/country/greec', 'www.study.eu/country/hungar', 'www.study.eu/country/icelan', 'www.study.eu/country/ital', 'www.study.eu/country/latvi', 'www.study.eu/country/liechtenstei', 'www.study.eu/country/lithuani', 'www.study.eu/country/malt', 'www.study.eu/country/netherland', 'www.study.eu/country/norwa', 'www.study.eu/country/polan', 'www.study.eu/country/romani', 'www.study.eu/country/spai', 'www.study.eu/country/switzerlan'], "[21877878, '*', '*', 60911675, 15278019, 13513295, 26589921, 8103940]": ['www.study.eu/university/university-west/child-and-youth-studies-1851', 'www.study.eu/university/university-west/internat

CALCOLO : PRECISION, RECALL E F1

In [4]:
import pickle

f_dominio_v = open(dominio+"_v_soap.pkl","rb")

denominatore_recall = 0
f = pickle.load(f_dominio_v)
denominatore_recall = len(f.keys())


f_cluster_dominio = open(dominio+"_clusters.pkl", "rb")
c = pickle.load(f_cluster_dominio)

print("\ncluster : {}".format(f_cluster_dominio.name))
denominatore_precision = 0
numeratore_precision = 0

for key, value in c.items():
  print(key,value)
  dizionario_precision = {}
  numeratore_precision_cluster = 0
  for link in value:
    if link.rsplit("/")[1] == "movie" :
      link = link.replace("/movie/","/tv/")
    if '/'.join(link.rsplit("/")[:2]) in dizionario_precision.keys():
      contatore = dizionario_precision.get('/'.join(link.rsplit("/")[:2]))
      contatore += 1
      dizionario_precision['/'.join(link.rsplit("/")[:2])] = contatore
    else :
      dizionario_precision['/'.join(link.rsplit("/")[:2])] = 1

  numeratore_precision_cluster += max(list(dizionario_precision.values()))
  print("Precision cluster {} : {}".format(key,numeratore_precision_cluster/len(value)))

  denominatore_precision += len(value)
  numeratore_precision += max(list(dizionario_precision.values()))



precision = numeratore_precision/denominatore_precision
recall = numeratore_precision/denominatore_recall
f1 = (2*precision*recall)/(precision+recall)
print("Precision {} : {} ; Recall {} : {} ; F1 : {}".format(dominio,precision,dominio,recall,f1))




cluster : study_clusters.pkl
[3601998, '*', 23187026, 42845222, '*', 915566, 32814414, 68809114] ['www.study.eu/country/austri', 'www.study.eu/country/belgiu', 'www.study.eu/country/czech-republi', 'www.study.eu/country/denmar', 'www.study.eu/country/finlan', 'www.study.eu/country/hungar', 'www.study.eu/country/irelan', 'www.study.eu/country/ital', 'www.study.eu/country/lithuani', 'www.study.eu/country/netherland', 'www.study.eu/country/norwa', 'www.study.eu/country/polan', 'www.study.eu/country/romani', 'www.study.eu/country/spai', 'www.study.eu/country/switzerlan', 'www.study.eu/country/united-kingdo', 'www.study.eu/article/study-zoolog', 'www.study.eu/article/what-is-the-ects-european-credit-transfer-and-accumulation-syste', 'www.study.eu/article/study-biomedicine-or-biomedical-science']
Precision cluster [3601998, '*', 23187026, 42845222, '*', 915566, 32814414, 68809114] : 0.8421052631578947
[42687545, 68651532, 23187026, 42845222, '*', '*', 32814414, 68809114] ['www.study.eu/coun